# Minimum Spanning Tree - Kruskal's Algorithm
https://opendsa-server.cs.vt.edu/ODSA/Books/CS3/html/Kruskal.html

### Table of Contents
- **[Kurslkal's Algorithm](#kruskal)**<br>

## Kruskals's Algorithm
- another algorithm for finding Minimum Spanning Tree (MST)
- also a greedy algorithm
    - makes the optimal choice at each step as it attempts to find the overall optimal way to solve the entire problem
- use [Parent Pointer Tree](./GeneralTreesUnionFind.ipynb) to find and join disjoint sets
- algorithm steps:
    1. partition the set of vertices into $V$ [disjoint sets](./GeneralTreesUnionFind.ipynb)
        - each set contains one vertex
    - process the edges in order of weight (sort, or use min heap priority queue)
    - if an edge connects two vertices in dfferent disjoint sets (FIND):
        - add the edge to the MST
        - combine the sets (UNION)

    - if the graph is connected, MST will have $|V|-1$ edges

<img src="./resources/Kruskals.png">

### visualize Kruskal's algorithm here: https://opendsa-server.cs.vt.edu/ODSA/Books/CS3/html/Kruskal.html

In [1]:
#include <iostream>
#include <vector>
#include <queue> // priority_queue
#include <climits> // sizes of integral types
#include <sstream>
#include <list>
#include <algorithm>

using namespace std;

using iPair = pair<int, int>; 

In [2]:
// general tree implementation for UNION/FIND
class ParPtrTree {
  private:
    vector<int> nodes; //nodes vector
    vector<int> weights; // weights for weighted union
  public:
    // constructor
    ParPtrTree(size_t size) {
        nodes.resize(size); //create nodes vector
        fill(nodes.begin(), nodes.end(), -1); // each node is its own root to start
        weights.resize(size); 
        fill(weights.begin(), weights.end(), 1);// set all base weights to 1
    }
    
    // Return the root of a given node with path compression
    // recursive algorithm that makes all ancestors of the current node
    // point to the root
    int FIND(int node) {
        if (nodes[node] == -1) return node;
        nodes[node] = FIND(nodes[node]);
        return nodes[node];
    }
    
    // Merge two subtrees if they are different
    void UNION(int node1, int node2) {
        int root1 = FIND(node1);
        int root2 = FIND(node2);
        // MERGE two trees
        if (root1 != root2) {
            if (weights[root1] < weights[root2]) {
                nodes[root1] = root2;
                weights[root2] += weights[root1];
            }
            else {
                nodes[root2] = root1;
                weights[root1] += weights[root2];
            }
        }    
    }
    
    string toString() {
        string parents = "[";
        string nodes = "[";
        for (int i=0; i < this->nodes.size(); i++) {
            parents += "\t" + to_string(this->nodes[i]);
            nodes += " \t" + to_string(i); 
        }
        parents += "\t]";
        nodes += "\t]";
        return parents + "\n" + nodes;
    }
};

In [3]:
// a structure to represent a weighted edge in graph 
struct Edge 
{ 
    int src, dest, weight; 
};

// custome Compare class to use with priority_queue (min heap)
struct Compare {
    bool operator()(const Edge& a, const Edge& b) 
    { 
        return a.weight > b.weight; 
    } 
};

## Representing Graph using Vector of Edge

In [4]:
// a structure to represent undirected 
// and weighted graph 
struct Graph 
{ 
    // V -> Number of vertices, E -> Number of edges 
    int V, E; 
    // graph is stored in a min heap priority_queue
    // Kruskal algo requires working with edges with smallest to highest weight 
    priority_queue<Edge, vector<Edge>, Compare > edges;
    // constructor
    Graph(int v, int e) {
        V = v;
        E = e;
    }
    
    void addEdge(int u, int v, int w) {
        edges.push({u, v, w});
    }
};

In [5]:
// function to construct MST using Kruskal's algorithm 
int KruskalMST(Graph& graph, vector<iPair> & MST) 
{ 
    if (graph.E == 0)
        return 0;

    int numMST = graph.V;  // initially V disjoint classes  
    ParPtrTree unionfind(graph.V);
    int weight = 0;

    while (numMST > 1 && !graph.edges.empty())
    { 
        // pick the smallest edge
        Edge edge = graph.edges.top();
        graph.edges.pop();
        int x = unionfind.FIND(edge.src); // root of src
        int y = unionfind.FIND(edge.dest); // root of dest
        // if src and dest nodes are in different sets
        if (x != y) 
        { 
            int u = edge.src;
            int v = edge.dest;
            // add weight
            weight += edge.weight;
            if (u > v) swap(u, v);
            // add u->v edge to MST
            MST.push_back({u, v});
            // combine equiv classes
            unionfind.UNION(u, v);
            numMST--; // one less MST
        } 
    } 
    return weight;
}

<img src="./resources/MCST.png">

In [6]:
// represent undirected graph shown in above diagram
// A->0, B->1, C->2, D->3, E->4, F->5
Graph graph(6, 8);
vector<iPair> MST;

In [7]:
// 16 bidirectional edges
graph.addEdge(0, 2, 7);
graph.addEdge(0, 4, 9);
graph.addEdge(1, 2, 5);
graph.addEdge(1, 5, 6);
graph.addEdge(2, 3, 1);
graph.addEdge(2, 5, 2);
graph.addEdge(3, 5, 2);
graph.addEdge(4, 5, 1);

In [8]:
int wt;

In [9]:
wt = KruskalMST(graph, MST);

In [10]:
cout << "total cost of MST = " << wt << endl;
cout << "MST edges:\n";
for(auto &p:MST)
    cout << p.first <<  "-" << p.second << '\n';


total cost of MST = 16
MST edges:
2-3
4-5
2-5
1-2
0-2


## Exercises
1. Solve Minimum Spanning Tree problem: https://open.kattis.com/problems/minspantree